# CIFAR 100 Dataset
To compare both types of networks we are going to use the CIFAR dataset, which contains 100 different object clases that the learnt networks should predict. Each sample of the dataset is a RGB image of size 32x32.

## Resnet
This model takes keras pretrained model.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar100
(_x_train, _y_train), (_x_test, _y_test) = cifar100.load_data(label_mode='fine')

/Users/soomin/.virtualenvs/tf/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
class_names = np.unique(_y_train)
num_classes = len(class_names)
# Data pre-processing
x_val = _x_train[40000:]
y_val = _y_train[40000:]
x_train = _x_train[:40000]
y_train = _y_train[:40000]
x_test = _x_test
y_test = _y_test

# Normalize the images to be values in range [0, 1]
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_val /= 255.0
x_test /= 255.0

from tensorflow.keras.utils import to_categorical
# Converts a class vector (integers) to binary class matrix
tr_labels = to_categorical(y_train, num_classes)
va_labels = to_categorical(y_val, num_classes)
te_labels = to_categorical(y_test, num_classes)

CIFAR_SIZE = (32, 32)

print("X training shape: ", x_train.shape)
print("Training label shape: ", tr_labels.shape)

print("X validation shape: ", x_val.shape)
print("Validation label shape: ", va_labels.shape)

print("X test shape: ", x_test.shape)
print("Test label shape: ", te_labels.shape)

X training shape:  (40000, 32, 32, 3)
Training label shape:  (40000, 100)
X validation shape:  (10000, 32, 32, 3)
Validation label shape:  (10000, 100)
X test shape:  (10000, 32, 32, 3)
Test label shape:  (10000, 100)


In [ ]:
# Tensorboard
from time import time
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))

# Train a convolutional Neuronal Network 

In [5]:
from tensorflow.keras import applications
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential,Model,load_model

input_shape = (CIFAR_SIZE[0], CIFAR_SIZE[1], 3)

res_base = applications.resnet50.ResNet50(weights=None, include_top=False, input_shape=input_shape)


In [7]:
res_output = res_base.output
res_output = GlobalAveragePooling2D()(res_output)
res_output = Dropout(0.7)(res_output)
prediction = Dense(num_classes, activation='softmax')(res_output)
model = Model(inputs=res_base.input, outputs=prediction)
model.summary()

In [9]:
from tensorflow.keras.optimizers import Adam
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, tr_labels, epochs=10, batch_size=16)

Epoch 1/10
 254/2500 [==>...........................] - ETA: 19:02 - loss: 9.9261 - accuracy: 0.0099       

KeyboardInterrupt: 

In [ ]:
from time import time

start = time()
loss, acc = model.evaluate(x_test, te_labels, verbose=0)
end = time()
print({'ETA': str(end - start), 'test_loss':str(loss), 'test_accuracy':str(acc)})